In [ ]:
import numpy as np

In [1]:
read=[]
f=open('readings.txt','r')
i=f.readline()
for i in f:
    if i!='\n':
        read.append(int(i.strip()))

In [2]:
read[-10:]

[-110, -55, -12, -29, -59, -73, -54, -28, 44, 178]

In [3]:
readstr=str(read)

In [4]:
readstr[:32]

'[48, 104, 90, 16, -32, -34, -9, '

In [5]:
readstr=readstr[1:-1]

In [6]:
sequences=[]
#prev=0
for i in range(len(readstr)-32):
    sequences.append(readstr[i:i+32])
    #prev=i

In [7]:
len(sequences)

955825

In [8]:
vocab=list(set(readstr))

In [9]:
vocab.sort()

In [10]:
(vocab)

[' ', ',', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [11]:
def to_one_hot(seq):
    ohseq=[]
    for i in str(seq):
        ohseq.append(get_one_hot(i))
    ohseq=np.array(ohseq)
    return ohseq 

In [ ]:
def get_one_hot(n):
    oh=np.zeros(len(vocab))
    oh[vocab.index(n)]=1
    return oh

In [ ]:
to_one_hot('45,23,-196').shape

In [ ]:
len('45,23,-196')

In [ ]:
#sequence_ohvs=[to_one_hot(i) for i in sequences]

In [ ]:
#sequence_ohvs=np.array(sequence_ohvs)

In [ ]:
#sequence_ohvs.shape

In [ ]:
sequences[:10]

In [12]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 128 
input_characters=vocab
#target_characters=vocab
m=len(sequences)
input_texts=sequences[:-int(m/10)-1]
target_texts=sequences[1:-int(m/10)]
target_texts=['\t'+i+'\n' for i in target_texts]
test_input_texts=sequences[-int(m/10)-1:-1]
test_target_texts=sequences[-int(m/10):]
input_characters = vocab #sorted(list(vocab))
num_encoder_tokens = len(vocab)
max_encoder_seq_length = 32
max_decoder_seq_length = 34

In [13]:
vocabd=list(set(readstr))
vocabd.append('\n')
vocabd.append('\t')
vocabd.sort()

In [14]:
vocabd

['\t', '\n', ' ', ',', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [15]:
target_characters=vocabd
num_decoder_tokens=len(vocabd)

In [16]:
#assert len(test_input_texts)==len(test_target_texts)

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [17]:
input_token_index = dict([(char, i) for i, char in enumerate(vocab)])
target_token_index = dict([(char, i) for i, char in enumerate(vocabd)])

In [ ]:
target_token_index['3']

# Converting to one hot type

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [ ]:
#encoder_input_test=sequence_ohvs[-int(m/10)-1:-1]
#decoder_input_test=sequence_ohvs[-int(m/10):]

In [ ]:
decoder_input_data.shape

In [ ]:
encoder_input_data.shape

In [ ]:
#np.array_equal(encoder_input_data,sequence_ohvs[:-1])

In [18]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.optimizers import Adam
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
# Run training
No train again
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=16,
          epochs=10,
          validation_split=0.2)


In [ ]:
model.save('nnts2_with_tab_enter.h5')

In [19]:
import keras
from keras.models import Model

In [20]:
model.load_weights('nnts2_with_tab_enter.h5')

In [21]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 13)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 15)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 128), (None, 72704       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 128),  73728       input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [22]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [23]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    #target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [24]:

for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    print(input_seq.shape)
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

NameError: name 'encoder_input_data' is not defined

In [ ]:
test_target_texts

In [ ]:
%who_ls

In [ ]:
test_input_texts[0]

In [27]:
import numpy as np
test_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
test_output_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [28]:
for i, (input_text, target_text) in enumerate(zip(test_input_texts,test_target_texts)):
    for t, char in enumerate(input_text):
        test_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        test_output_data[i, t, target_token_index[char]] = 1.

In [29]:
correct=0
wrong=0
for i in range(len(test_input_data)-2):
    ip=test_input_data[i:i+1]
    op=decode_sequence(ip)
    if np.array_equal(test_input_data[i+1:i+2],op):
        correct=correct+1
    else:
        wrong=wrong+1
    #print(ip)
    print(op)
    #break

103, -119, -51, -13, -34, -65, -

03, -119, -51, -15, -34, -65, -59

3, -119, -51, -15, -34, -65, -59

, -119, -51, -15, -34, -65, -59,

 -119, -51, -15, -34, -65, -59, 

-119, -51, -15, -34, -65, -59, -

119, -51, -15, -34, -65, -59, -3

19, -51, -15, -34, -65, -59, -38

9, -51, -15, -34, -65, -59, -38,

, -51, -15, -34, -65, -59, -38, 

 -51, -15, -34, -65, -59, -38, -

-51, -15, -34, -65, -59, -38, -18

51, -15, -34, -65, -59, -38, -18,

1, -15, -34, -65, -59, -38, -18,

, -15, -34, -65, -59, -38, -18, 4

 -15, -34, -65, -59, -38, -18, 42

-15, -34, -65, -59, -38, -18, 42

15, -34, -65, -59, -38, -18, 42,

5, -34, -65, -59, -38, -18, 42, 

, -34, -65, -59, -38, -18, 42, 

 -34, -65, -59, -38, -18, 42, 1

-34, -65, -59, -38, -18, 42, 194

34, -65, -59, -38, -18, 42, 194,

4, -65, -59, -38, -18, 42, 194, 

, -65, -59, -38, -18, 42, 194, 2

 -65, -59, -38, -18, 42, 194, 2

-65, -59, -38, -18, 42, 194, 206

65, -59, -38, -18, 42, 194, 206,

5, -59, -38, -18, 42, 194, 206, 

, -59, -38, 

37, 195, 82, -45, -99, -75, -24,

7, 195, 82, -45, -99, -75, -24, 

, 195, 82, -45, -99, -75, -24, 

 195, 82, -45, -99, -75, -24, -

195, 82, -45, -99, -75, -24, -24

95, 82, -45, -99, -75, -24, -24,

5, 82, -45, -99, -75, -24, -24, 

, 82, -45, -99, -75, -24, -24, 

 82, -45, -99, -75, -24, -24, -

82, -45, -99, -75, -24, -24, -46

2, -45, -99, -75, -24, -24, -46

, -45, -99, -75, -24, -24, -46,

 -45, -99, -75, -24, -24, -46, 

-45, -99, -75, -24, -24, -46, -

45, -99, -75, -24, -24, -46, -64

5, -99, -75, -24, -24, -46, -64

, -99, -75, -24, -24, -46, -64, 

 -99, -75, -24, -24, -46, -64, -

-99, -75, -24, -24, -46, -64, -

99, -75, -24, -24, -46, -64, -41

9, -75, -24, -24, -46, -64, -41,

, -75, -24, -24, -46, -64, -41, 

 -75, -24, -24, -46, -64, -41, 

-74, -22, -24, -46, -64, -41, -15,

75, -24, -24, -46, -64, -41, -15

5, -24, -24, -46, -64, -41, -15,

, -24, -24, -46, -64, -41, -15, 

 -24, -24, -46, -64, -41, -15, -

-24, -24, -46, -64, -41, -15, -9,

24, -24, -46, -64, -41

49, -58, -39, -19, -8, -4, 77, 

9, -58, -39, -19, -8, -4, 77, 17

, -58, -39, -19, -8, -4, 77, 17

 -58, -39, -19, -8, -4, 77, 179

-58, -39, -19, -8, -4, 77, 179,

58, -39, -19, -8, -4, 77, 179,

8, -39, -19, -8, -4, 77, 179, 1

, -62, -19, -9, 39, 187, 208, 

 -39, -19, -8, -4, 77, 179, 138,

-39, -19, -8, -4, 77, 179, 138,

39, -19, -8, -4, 77, 179, 138, 1

9, -19, -8, -4, 77, 179, 138, 1

, -19, -8, -4, 77, 179, 138, 16, 

 -19, -8, -4, 77, 179, 138, 16, -

-19, -8, -4, 77, 179, 138, 16, -

19, -8, -4, 77, 179, 138, 16, -64

9, -8, -4, 77, 179, 138, 16, -64,

, -8, -4, 77, 179, 138, 16, -64,

 -8, -4, 77, 179, 138, 16, -64, 

-8, -4, 77, 179, 138, 16, -64, -

8, -4, 77, 179, 138, 16, -64, -6

, 4, 77, 179, 138, 16, -64, -69,

 -4, 77, 179, 138, 16, -64, -69

-4, 77, 179, 138, 16, -64, -69,

4, 77, 179, 138, 16, -64, -69, 

, 77, 179, 138, 16, -64, -69, -

 77, 179, 138, 16, -64, -69, -2

77, 195, 162, -67, -64, -93, -

7, 179, 138, 16, -64, -69, -25, 

, 179, 138, 16, -64, -69, -2

, -104, -88, -25, -8, -42, -66,

 -104, -88, -25, -8, -42, -66, 

-104, -88, -25, -8, -42, -66, -4

104, -88, -25, -8, -42, -66, -47

04, -88, -25, -9, -43, -68, -7

4, -88, -25, -8, -42, -66, -47,

, -88, -25, -8, -42, -66, -47, 

 -88, -25, -8, -42, -66, -47, -

-88, -25, -8, -42, -66, -47, -34

88, -25, -8, -42, -66, -47, -34,

8, -25, -8, -42, -66, -47, -34,

, -25, -8, -42, -66, -47, -34, -

 -25, -8, -42, -66, -47, -34, -1

-25, -8, -42, -66, -47, -34, -13

25, -8, -42, -66, -47, -34, -13,

5, -2, -43, -66, -49, -32, -18, 

, -8, -42, -66, -47, -34, -13, 

 -8, -42, -66, -47, -34, -13, 59

-8, -42, -66, -47, -34, -13, 59,

8, -42, -66, -47, -34, -13, 59, 

, -42, -66, -47, -34, -13, 59, 2

 -42, -66, -47, -34, -13, 59, 20

-42, -66, -47, -34, -13, 59, 208

42, -66, -47, -34, -13, 59, 208,

2, -66, -47, -34, -13, 59, 208, 

, -66, -47, -34, -13, 59, 208, 1

 -66, -47, -34, -13, 59, 208, 17

-66, -47, -34, -13, 59, 208, 17

66, -47, -34, -13, 59, 208, 177

6, -47, -34, -13, 59, 208

, 186, 22, -109, -110, -36, -7, 

 186, 22, -109, -110, -36, -7, -

186, 22, -109, -110, -36, -7, -3

86, 22, -109, -110, -36, -7, -35

6, 22, -109, -110, -36, -7, -35,

, 22, -109, -110, -36, -7, -35, 

 22, -109, -110, -36, -7, -35, -

22, -109, -110, -36, -7, -35, -6

2, -109, -110, -36, -7, -35, -63

, -109, -110, -36, -7, -35, -63

 -109, -110, -36, -7, -35, -63,

-109, -110, -36, -7, -35, -63, -

109, -110, -36, -7, -35, -63, -5

09, -112, -43, -16, -37, -63, -5

9, -110, -36, -7, -35, -63, -55

, -110, -36, -7, -35, -63, -55,

 -110, -36, -7, -35, -63, -55, 

-110, -36, -7, -35, -63, -55, -

110, -36, -7, -35, -63, -55, -3

10, -36, -7, -35, -63, -55, -39

0, -36, -7, -35, -63, -55, -39,

, -36, -7, -35, -63, -55, -39, 

 -36, -7, -35, -63, -55, -39, -

-36, -7, -35, -63, -55, -39, -12

36, -7, -35, -63, -55, -39, -12,

6, -7, -35, -63, -55, -39, -12, 

, -7, -35, -63, -55, -39, -12, 

 -7, -35, -63, -55, -39, -12, 74

-7, -35, -63, -55, -39, -12, 74,

7, -35, -63, -55, -39, -12

...

68, -5, -11, -45, -69, -58, -33,

8, -5, -11, -45, -69, -58, -33, 

, -5, -11, -45, -69, -58, -33, 4

 -5, -11, -45, -69, -58, -33, 42,

-5, -11, -45, -69, -58, -33, 42,

5, -11, -45, -69, -58, -33, 42, 

, -11, -45, -69, -58, -33, 42, 1

 -11, -45, -69, -58, -33, 42, 19

-11, -45, -69, -58, -33, 42, 192

11, -45, -69, -58, -33, 42, 192, 

1, -45, -69, -58, -33, 42, 192, 

, -45, -69, -58, -33, 42, 192, 2

 -45, -69, -58, -33, 42, 192, 2

-45, -69, -58, -33, 42, 192, 203

45, -69, -58, -33, 42, 192, 203

5, -69, -58, -33, 42, 192, 203, 

, -69, -58, -33, 42, 192, 203, 3

 -69, -58, -33, 42, 192, 203, 36

-69, -58, -33, 42, 192, 203, 36,

69, -58, -33, 42, 192, 203, 36, 

9, -58, -33, 42, 192, 203, 36, 

, -58, -33, 42, 192, 203, 36, -

 -58, -33, 42, 192, 203, 36, -10

-58, -33, 42, 192, 203, 36, -10

58, -33, 42, 192, 203, 36, -101

8, -33, 42, 192, 203, 36, -101, 

, -33, 42, 192, 203, 36, -101, -

 -33, 42, 192, 203, 36, -101, -1

-33, 42, 192, 203, 36, -101, -11

3, 42, 192, 230, 3

125, -113, -39, -12, -41, -64, -

25, -113, -39, -12, -41, -64, -

5, -113, -39, -12, -41, -64, -60

, -113, -39, -12, -41, -64, -60

 -113, -39, -12, -41, -64, -60, 

-113, -39, -12, -41, -64, -60, -

113, -39, -12, -41, -64, -60, -3

13, -34, -12, -41, -64, -60, -3

3, -43, -12, -41, -64, -60, -32, 

, -39, -12, -41, -64, -60, -32, 

 -39, -12, -41, -64, -60, -32, -

-39, -12, -41, -64, -60, -32, -14

39, -12, -41, -64, -60, -32, -12

9, -12, -41, -64, -60, -32, -12,

, -12, -41, -64, -60, -32, -12, 2

 -12, -41, -64, -60, -32, -12, 2

-12, -41, -64, -60, -32, -12, 29

12, -41, -64, -60, -32, -12, 29,

2, -41, -64, -60, -32, -12, 29, 

, -41, -64, -60, -32, -12, 29, 1

 -44, -67, -63, -32, -19, 28, 193

-44, -67, -63, -31, -2, 120, 236,

41, -64, -60, -32, -12, 29, 180,

1, -46, -59, -31, -12, 2, 104, 2

, -64, -60, -32, -12, 29, 180, 2

 -64, -60, -32, -12, 29, 180, 2

-64, -60, -32, -12, 29, 180, 22

64, -60, -32, -12, 29, 180, 227,

4, -60, -32, -12, 29, 180, 227,

, -60, -32, -12

7, 150, 234, 107, -59, -120, -8

, 150, 234, 107, -59, -120, -80,

 150, 234, 107, -59, -120, -80, 

150, 234, 107, -59, -120, -80, -

50, 243, 108, -47, -119, -87, -32

0, 234, 107, -59, -120, -80, -19

, 234, 107, -59, -120, -80, -19,

 234, 107, -59, -120, -80, -19,

234, 107, -59, -120, -80, -19, -

34, 107, -59, -120, -80, -19, -2

4, 107, -59, -120, -80, -19, -23

, 107, -59, -120, -80, -19, -23,

 107, -59, -120, -80, -19, -23, 

107, -59, -120, -80, -19, -23, -

07, -59, -120, -80, -19, -23, -

7, -59, -120, -80, -19, -23, -5

, -59, -120, -80, -19, -23, -54,

 -59, -120, -80, -19, -23, -54, 

-59, -120, -80, -19, -23, -54, -

59, -120, -80, -19, -23, -54, -

9, -120, -80, -19, -23, -54, -7

, -120, -80, -19, -23, -54, -76,

 -120, -80, -19, -23, -54, -76, 

-120, -80, -19, -23, -54, -76, 

120, -80, -19, -23, -54, -76, -

20, -80, -19, -23, -54, -76, -51

0, -80, -19, -23, -54, -76, -51,

, -80, -19, -23, -54, -76, -51,

 -80, -19, -23, -54, -76, -51, -

-80, -19, -23, -54, -7

, -46, -68, -60, -32, -14, 82, 

 -46, -68, -60, -32, -14, 82, 22

-46, -68, -60, -32, -14, 82, 223

46, -68, -60, -32, -14, 82, 223,

6, -68, -60, -32, -14, 82, 223,

, -68, -60, -32, -14, 82, 223, 

 -68, -60, -32, -14, 82, 223, 1

-68, -60, -32, -14, 82, 223, 17

68, -60, -32, -14, 82, 223, 170

8, -60, -32, -14, 82, 223, 170,

, -60, -32, -14, 82, 223, 170, 4

 -60, -32, -14, 82, 223, 170, 4

-60, -32, -14, 82, 223, 170, 4,

60, -32, -14, 82, 223, 170, 4, -

0, -32, -14, 82, 223, 170, 4, -1

, -32, -14, 82, 223, 170, 4, -10

 -32, -14, 82, 223, 170, 4, -106

-32, -14, 82, 223, 170, 4, -106,

32, -14, 82, 223, 170, 4, -106, 

2, -14, 82, 223, 170, 4, -106, -

, -14, 82, 223, 170, 4, -106, -1

 -14, 25, 203, 186, 22, -109, -1

-14, 25, 203, 186, 22, -109, -110

14, 28, 172, 230, 82, -79, -126,

4, 82, 223, 170, 4, -106, -108, 

, 82, 223, 170, 4, -106, -108, -

 82, 223, 170, 4, -106, -108, -4

82, 223, 170, 4, -106, -108, -4

2, 223, 170, 4, -106, -108, -49,

, 223, 170, 4, -106, -1

KeyboardInterrupt: 